In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.externals import joblib

In [4]:
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load data from file

In [6]:
merged_data = pd.read_pickle('merged_data_pct_change.pkl')

# Create train data and test data

In [7]:
X = merged_data['rawText']
y = merged_data['1day relative pct change']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Tf-Idf

### Load from file

In [9]:
#tfidf = joblib.load('tfidf_lemmatokenizer__maxdf0_4__mindf300.pkl')

In [10]:
#tfidf_vectors = tfidf.transform(X_train)

### Recreate vectorizer from scratch

In [9]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t.lower(), 'v') for t in word_tokenize(doc)]

In [11]:
tfidf = TfidfVectorizer(max_df=0.4, min_df=300, ngram_range=(1,2),tokenizer=LemmaTokenizer(), stop_words='english')

In [12]:
tfidf_vectors = tfidf.fit_transform(X_train)

In [14]:
#joblib.dump(tfidf, 'tfidf_lemmatokenizer__maxdf0_4__mindf300.pkl')

# DNN model

In [13]:
dense_tfidf_vectors = tfidf_vectors.toarray()

In [52]:
# define base model
def baseline_model():
    model = Sequential()
    model.add(Dense(5000, input_dim=dense_tfidf_vectors.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(1000, kernel_initializer='normal', activation='relu'))
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [53]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=10, verbose=2)

In [18]:
'''
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, dense_tfidf_vectors, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
'''

'\nkfold = KFold(n_splits=10)\nresults = cross_val_score(estimator, dense_tfidf_vectors, y_train, cv=kfold)\nprint("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))\n'

In [54]:
estimator.fit(dense_tfidf_vectors, y_train)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[7494,5000]
	 [[Node: dense_11/random_normal/RandomStandardNormal = RandomStandardNormal[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=8605955, _device="/job:localhost/replica:0/task:0/gpu:0"](dense_11/random_normal/shape)]]

Caused by op 'dense_11/random_normal/RandomStandardNormal', defined at:
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
   

In [ ]:
test_vectors = tfidf.transform(X_test)

In [ ]:
test_vectors = test_vectors.toarray()

In [ ]:
y_predicted = estimator.predict(test_vectors)

In [ ]:
plt.figure(figsize=(20,10))
sns.tsplot(y_test[130:150], color='r')
sns.tsplot(y_predicted[130:150])

In [ ]:
plt.figure(figsize=(20,10))
#sns.tsplot(y_test[0:50])
error = y_test-y_predicted
sns.tsplot(error[0:50], color='r')

In [ ]:
error_value = mean_squared_error(y_test, y_predicted)

In [ ]:
error_value

In [ ]:
y_test_pos = y_test > 0

In [ ]:
y_predicted_pos = y_predicted > 0

In [ ]:
diff = y_test_pos != y_predicted_pos

In [ ]:
y_predicted_pos[0:5]

In [ ]:
y_test_pos[0:5]

In [ ]:
diff[0:5]

In [ ]:
sns.countplot(diff)